In [1]:
import pandas as pd 
import plotly.express as px

In [2]:
TYPES = [
    0, # Асфальт
    1, # Плитка
    2, # Грунт
]

In [3]:
gyro_df = pd.read_csv(r'..\data\raw_data\ground-2024-04-07_15-01-24')
gyro_df["gx"] = gyro_df['x']
gyro_df["gy"] = gyro_df['y']
gyro_df["gz"] = gyro_df['z']
gyro_df = gyro_df.drop(columns=['x', 'y', 'z'])
gyro_df.head(10)

,time,seconds_elapsed,gx,gy,gz
0,1712502084371039500,0.084040,-0.036956,0.040469,-0.016340
1,1712502084380549600,0.093550,-0.035735,0.039247,-0.019547
2,1712502084390059800,0.103060,-0.036345,0.039247,-0.021227
3,1712502084399570000,0.112570,-0.036498,0.040469,-0.026266
4,1712502084409080000,0.122080,-0.036498,0.036804,-0.026266
5,1712502084418590200,0.131590,-0.033444,0.032375,-0.025045
6,1712502084428100400,0.141100,-0.030390,0.029321,-0.023823
7,1712502084437610500,0.150611,-0.024892,0.023670,-0.020311
8,1712502084447120600,0.160121,-0.021838,0.020463,-0.016646
9,1712502084456630800,0.169631,-0.017562,0.018020,-0.014813


In [4]:
acc_df = pd.read_csv(r'..\data\raw_data\ground-2024-04-07_15-01-24')
acc_df["ax"] = acc_df['x']
acc_df["ay"] = acc_df['y']
acc_df["az"] = acc_df['z']
acc_df = acc_df.drop(columns=['x', 'y', 'z'])
acc_df.head(10)


,time,seconds_elapsed,ax,ay,az
0,1712502084438730500,0.151730,-0.072700,-0.034584,0.039080
1,1712502084457750800,0.170751,-0.045523,-0.040381,0.022512
2,1712502084476771000,0.189771,-0.021126,-0.039886,0.031643
3,1712502084495791400,0.208791,-0.036058,-0.037888,0.029950
4,1712502084514811400,0.227811,-0.042389,-0.025120,0.027601
5,1712502084533831700,0.246832,-0.030970,0.003347,0.022793
6,1712502084552852000,0.265852,0.003649,0.045678,0.008373
7,1712502084571872300,0.284872,0.018996,0.092182,0.017884
8,1712502084590892500,0.303893,0.032115,0.127484,0.001069
9,1712502084609917200,0.322917,0.045767,0.143211,0.015689


In [5]:
df = pd.merge_asof(acc_df, gyro_df, on="seconds_elapsed", direction="nearest")
df["time"] = df["time_x"]
df["class"] = 1
df = df.drop(columns=["time_x", "time_y"])
df.head(10)

,seconds_elapsed,ax,ay,az,gx,gy,gz,time,class
0,0.151730,-0.072700,-0.034584,0.039080,-0.024892,0.023670,-0.020311,1712502084438730500,1
1,0.170751,-0.045523,-0.040381,0.022512,-0.017562,0.018020,-0.014813,1712502084457750800,1
2,0.189771,-0.021126,-0.039886,0.031643,-0.013897,0.014202,-0.008094,1712502084476771000,1
3,0.208791,-0.036058,-0.037888,0.029950,-0.009621,0.007941,-0.003360,1712502084495791400,1
4,0.227811,-0.042389,-0.025120,0.027601,-0.002443,0.002443,-0.002749,1712502084514811400,1
5,0.246832,-0.030970,0.003347,0.022793,0.004276,-0.001985,0.000764,1712502084533831700,1
6,0.265852,0.003649,0.045678,0.008373,0.006108,-0.005650,0.005039,1712502084552852000,1
7,0.284872,0.018996,0.092182,0.017884,0.006872,-0.007025,0.008094,1712502084571872300,1
8,0.303893,0.032115,0.127484,0.001069,0.005650,-0.005192,0.006261,1712502084590892500,1
9,0.322917,0.045767,0.143211,0.015689,0.004887,-0.002596,0.003207,1712502084609917200,1


In [6]:
df_cut = df[(df["seconds_elapsed"] > 5) & (df["seconds_elapsed"] < (df["seconds_elapsed"].max() - 5))]
fig = px.line(df_cut, x="seconds_elapsed", y=["ax", "ay", "az"])
fig.show()

In [7]:
df_cut = df[(df["seconds_elapsed"] > 5) & (df["seconds_elapsed"] < (df["seconds_elapsed"].max() - 5))]
fig = px.line(df_cut, x="seconds_elapsed", y=["gx", "gy", "gz"])
fig.show()

In [12]:
import os

SAVE_PATH = '..\preprocessed_data'


def preproccess_data(data_path):
    types_dict = {
        "asphalt": 0,  # Асфальт
        "tiles": 1,  # Плитка
        "crumb": 2,  # крошка
        "stones": 3,  # брусчатка
    }

    gyro_df = pd.read_csv(data_path)
    gyro_df["gx"] = gyro_df['x']
    gyro_df["gy"] = gyro_df['y']
    gyro_df["gz"] = gyro_df['z']
    gyro_df = gyro_df.drop(columns=['x', 'y', 'z'])

    acc_df = pd.read_csv(data_path)
    acc_df["ax"] = acc_df['x']
    acc_df["ay"] = acc_df['y']
    acc_df["az"] = acc_df['z']
    acc_df = acc_df.drop(columns=['x', 'y', 'z'])

    loc_df = pd.read_csv(data_path)
    loc_df = loc_df.drop(columns=['bearingAccuracy', 'speedAccuracy',
                         'verticalAccuracy', 'horizontalAccuracy', 'speed', 'bearing', 'altitude',])

    df = pd.merge_asof(acc_df, gyro_df, on="seconds_elapsed",
                       direction="nearest")
    df = pd.merge_asof(df, loc_df, on="seconds_elapsed",
                       direction="nearest")
    df["time"] = df["time_x"]
    df = df.drop(columns=["time_x", "time_y"])
    cl = None
    for item in types_dict:
        if item in data_path:
            cl = types_dict[item]
            break
    if cl == None:
        print("couldn't find class")
    df["class"] = cl
    save_file_path = os.path.join(
        SAVE_PATH, os.path.basename(data_path)+'.csv')
    df.to_csv(save_file_path, index=False)
    print(f"OK – {save_file_path}")

In [13]:
for data_path in [os.path.join(r"..\raw_data\surfaces", path) for path in os.listdir(r"..\raw_data\surfaces")]:
    print(f"Trying {data_path}")
    try:
        preproccess_data(data_path)
    except Exception as e:
        print(f'Got error with {data_path}:', e)

Trying ..\raw_data\surfaces\asphalt-2024-06-15_16-07-55
OK – ..\preprocessed_data\asphalt-2024-06-15_16-07-55.csv
Trying ..\raw_data\surfaces\asphalt-2024-06-15_16-13-20
OK – ..\preprocessed_data\asphalt-2024-06-15_16-13-20.csv
Trying ..\raw_data\surfaces\asphalt-2024-06-15_16-16-43
OK – ..\preprocessed_data\asphalt-2024-06-15_16-16-43.csv
Trying ..\raw_data\surfaces\asphalt-2024-06-15_16-17-44
OK – ..\preprocessed_data\asphalt-2024-06-15_16-17-44.csv
Trying ..\raw_data\surfaces\asphalt-2024-06-15_16-18-42
OK – ..\preprocessed_data\asphalt-2024-06-15_16-18-42.csv
Trying ..\raw_data\surfaces\asphalt-2024-06-15_16-19-31
OK – ..\preprocessed_data\asphalt-2024-06-15_16-19-31.csv
Trying ..\raw_data\surfaces\asphalt-2024-06-15_16-30-49
OK – ..\preprocessed_data\asphalt-2024-06-15_16-30-49.csv
Trying ..\raw_data\surfaces\asphalt-2024-06-15_16-31-38
OK – ..\preprocessed_data\asphalt-2024-06-15_16-31-38.csv
Trying ..\raw_data\surfaces\asphalt-2024-06-15_16-32-45
OK – ..\preprocessed_data\asphal

In [13]:
import json

[os.path.join(r"..\data\preprocessed_data", path) for path in os.listdir(r"..\data\preprocessed_data")]

['..\\data\\preprocessed_data\\asphalt_-2024-04-07_14-57-43.csv',
 '..\\data\\preprocessed_data\\asphalt_-2024-04-07_15-02-50.csv',
 '..\\data\\preprocessed_data\\ground-2024-04-07_14-53-13.csv',
 '..\\data\\preprocessed_data\\ground-2024-04-07_15-01-24.csv',
 '..\\data\\preprocessed_data\\tile-2024-04-07_14-52-23.csv',
 '..\\data\\preprocessed_data\\tile-2024-04-07_14-59-03.csv']